## Mission 3

이번 미션에서는 Figure S2A 아랫쪽에 표시되는 clustered error 분석 (CIMS; Crosslinking-induced mutation sites)을 합니다. 자외선에 의해서 단백질이 붙어있다가 단백질 분해효소로 단백질을 뜯어내고 남은 자리에서는 reversetranscriptase가 오류를 자주 냅니다. 오류율을 분석해서 단백질이 앉아 있었던 자리를 분석해 봅시다.

데이터는 지난 번과 동일하기 때문에 이미 다운로드 받으셨다면 다시 다운로드 받을 필요는 없습니다.

In [1]:
%cd data

/data/project/danyoung/bioinfo1-project/data


Genome 전체로 계산하는 방법은 자원도 꽤 쓰고 시간이 걸리기 때문에 여기서는 유전자 단위로 간략하게 하도록 합시다. 예시로 Mirlet7g를 해 보고, 추가로 Mirlet7f-1과 Mirlet7d도 봐서 비교해 봅시다.

논문은 예전 레퍼런스 유전체인 mm9을 쓰고 있기 때문에, 현재 사용되는 레퍼런스 mm39와는 좌표계에 차이가 있습니다. 새로 annotation에서 위치를 찾습니다.

In [2]:
!grep -i mirlet7g gencode.gtf

chr9	ENSEMBL	gene	106056039	106056126	.	+	.	gene_id "ENSMUSG00000065440.3"; gene_type "miRNA"; gene_name "Mirlet7g"; level 3; mgi_id "MGI:2676800";
chr9	ENSEMBL	transcript	106056039	106056126	.	+	.	gene_id "ENSMUSG00000065440.3"; transcript_id "ENSMUST00000083506.3"; gene_type "miRNA"; gene_name "Mirlet7g"; transcript_type "miRNA"; transcript_name "Mirlet7g-201"; level 3; transcript_support_level "NA"; mgi_id "MGI:2676800"; tag "basic";
chr9	ENSEMBL	exon	106056039	106056126	.	+	.	gene_id "ENSMUSG00000065440.3"; transcript_id "ENSMUST00000083506.3"; gene_type "miRNA"; gene_name "Mirlet7g"; transcript_type "miRNA"; transcript_name "Mirlet7g-201"; exon_number 1; exon_id "ENSMUSE00000522665.2"; level 3; transcript_support_level "NA"; mgi_id "MGI:2676800"; tag "basic";


Splicing 되지 않는 transcript이기 때문에 결과가 단조롭습니다. 해당 좌표 범위로 bam을 솎아냅니다.

In [3]:
!samtools view -b -o CLIP-let7g.bam CLIP-35L33G.bam chr9:106056039-106056126
!samtools view CLIP-let7g.bam | wc -l

163


In [12]:
!samtools view CLIP-let7g.bam | tail

SRR458758.18228268	0	chr9	106056095	40	3S27M	*	0	0	ACAGAGATAACTGTACAGGCCACTGCCTTG	IIIIIIIIIHIIIIIIIIIIBIIIIIIIII	MD:Z:27	NH:i:1	HI:i:1	NM:i:0	SM:i:40	XQ:i:40	X2:i:0	XO:Z:UU
SRR458758.19653091	0	chr9	106056095	39	1S42M	*	0	0	AGAGATAACTGTACAGGCCACTGCCTTGCCAGGAACAGTGCAC	HHHHHEHHHHHHHHH>BGBGGFGDG@BEAEFHHHHHHF<E8AC	MD:Z:42	NH:i:1	HI:i:1	NM:i:0	SM:i:39	XQ:i:40	X2:i:0	XO:Z:UU	XS:A:-
SRR458758.22527245	0	chr9	106056095	39	28M	*	0	0	GAGATAACTGTACAGGCCACTGCCTTGC	HGHHHGDHHHGGGEGGGDDD>DGG<@?D	MD:Z:28	NH:i:1	HI:i:1	NM:i:0	SM:i:39	XQ:i:40	X2:i:0	XO:Z:UU	XS:A:+	XY:Z:+ENSMUST00000083506.3:57..84
SRR458758.27842792	0	chr9	106056095	39	1S42M	*	0	0	AGAGATAACTGTACAGGCCACTGCCTTGCCAGGAACAGTGCAC	GGBG@HHEGHHEHHDC=>EFDGG@GHHGFHHDGFGFEGBBGEG	MD:Z:42	NH:i:1	HI:i:1	NM:i:0	SM:i:39	XQ:i:40	X2:i:0	XO:Z:UU	XS:A:-
SRR458758.30198532	0	chr9	106056095	35	3S27M	*	0	0	ACAGAGATAACTGTACAGGCCACTGCCTTG	DBDDDBD@DDBD<1D?@*3@DDDBDDDDD@	MD:Z:27	NH:i:1	HI:i:1	NM:i:0	SM:i:35	XQ:i:40	X2:i:0	XO:Z:UU
SRR458758.10618192	0	chr9	10605609

각 자리에 mapping된 read들의 분포를 요약하기 위해 samtools mpileup을 이용합니다. Shannon entropy를 구할 때는 reference를 따로 줄 필요는 없습니다. samtools를 쓰지 않고 직접 read를 분석하면 더욱 더 재미있습니다.

In [13]:
!samtools mpileup CLIP-let7g.bam > CLIP-let7g.pileup
!wc -l CLIP-let7g.pileup

[mpileup] 1 samples in 1 input files
68548 CLIP-let7g.pileup


Mirlet7g는 100bp도 안 되는 짧은 gene인데 `mpileup` 결과는 엄청 길게 나옵니다. 왜 그럴까요? (잘 한 번 생각해 봅시다)

이제 Mirlet7g의 핵심부분을 살펴봐서 어떻게 하면 좋을지 견적을 내 봅니다.

In [27]:
!head CLIP-let7g.pileup

chr9	106007092	N	9	^Ga^Ia^Ha^Ia^Ga^Ga^Ha^Ia^Ia	<IFIGGHII
chr9	106007093	N	9	ggggggggg	AFGIEGDII
chr9	106007094	N	9	ccccccccc	EHHIGAFEH
chr9	106007095	N	9	aaaaaaaaa	?HDIFA>II
chr9	106007096	N	9	aaaaaaaaa	EHHIG@CHH
chr9	106007097	N	9	ttttttttt	=HGEGDFII
chr9	106007098	N	9	aaaaaaaaa	?HHIGDEII
chr9	106007099	N	9	g-2nng-2nng-2nng-2nng-2nng-2nng-2nng-2nng-2nn	?IEHGEDII
chr9	106007100	N	9	*********	8IBIBC@EI
chr9	106007101	N	9	*********	8IBIBC@EI


In [28]:
!tail CLIP-let7g.pileup

chr9	106075630	N	9	aaaaaaaaa	GIHIGGHII
chr9	106075631	N	9	ggggggggg	=IFIE?GII
chr9	106075632	N	9	ggggggggg	4HGIGCBHI
chr9	106075633	N	9	ggggggggg	<IFIDADIH
chr9	106075634	N	9	ccccccccc	?EEIDEGII
chr9	106075635	N	9	ttttttttt	:GGFECDII
chr9	106075636	N	9	ccccccccc	4IDIB?CGI
chr9	106075637	N	9	aaaaaaaaa	7IGIGBCGI
chr9	106075638	N	9	ttttttttt	7GGIGDEGI
chr9	106075639	N	9	t$t$t$t$t$t$t$t$t$	>IGFDDDGH


In [15]:
!awk '$2 >= 106056039 && $2 <= 106056126 { print $0; }' CLIP-let7g.pileup > CLIP-let7g-gene.pileup
!tail CLIP-let7g-gene.pileup

chr9	106056117	N	138	<<<<<<<<<CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC	BHEIG?DIIHGHII:;HGIIDGGGIIBGEB?>H<GGIIIIDIIGFHIHH9IGG=GGHIDH?DGIBIIHHGGIEIGI8GIDHDG.GGHEDIIDIDGDHIDGIFGHG;DCDDHEE@I?CGG:IHGIBGIHIIDG@DHIGG
chr9	106056118	N	139	<<<<<<<<<CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC	BHEIG?DIIHGHII3AHIIIDGGGIIB3EHFBH>GIIIIIIIIIHIIHHAIIGBDGHIDH?GIIGIIHHDIIHIGIGGI=BGG9GIHHHIIGI=IAIIGGIEHGIEBG@DHDE;IBDGG4IGGIGGIIIIGGGDGIH1G
chr9	106056119	N	138	<<<<<<<<<TTTTTTTTTTTTTTTTTTTTTTTTTT$TTTT$TTTTTTTTTTTTTTT$TTTT$TTTTTTT$TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT	BHEIG?DIIGEFII:/GIIEDDHDIIF>BH3<HBEGIHIGFIIIBIIBE:HHGD>EGGDH7BIIGGIHHDHIHDEIGEIDHGG;GIHGHHIGD:IGHD:GGEHH?3G87HD</I=DGG=IHHIHBIFII@<HDHHH1G
chr9	106056120	N	133	<<<<<<<<<TTTTTTTTTTTTTTTTTTTTTTTTTTTT$TTTT$TTT$TTTTTTTTTT$TTTTTTTT$TTTTTTTT

In [30]:
!head CLIP-let7g-gene.pileup

chr9	106056039	N	9	<<<<<<<<<	BHEIG?DII
chr9	106056040	N	9	<<<<<<<<<	BHEIG?DII
chr9	106056041	N	9	<<<<<<<<<	BHEIG?DII
chr9	106056042	N	9	<<<<<<<<<	BHEIG?DII
chr9	106056043	N	9	<<<<<<<<<	BHEIG?DII
chr9	106056044	N	9	<<<<<<<<<	BHEIG?DII
chr9	106056045	N	32	<<<<<<<<<^HT^DT^IT^GT^HT^HT^IT^GT^IT^HT^HT^IT^IT^HT^HT^GT^HT^GT^HT^HT^HT^IT^GT	BHEIG?DIIG9GEGGIDGEGFGBG;DDGHDGG
chr9	106056046	N	32	<<<<<<<<<GGGGGGGGGGGGGGGGGGGGGGG	BHEIG?DIIG<GGGGIGGBGFG?G?GEGHHGG
chr9	106056047	N	32	<<<<<<<<<AAAAAAAAAAAAAAAAAAAAAAA	BHEIG?DIIG7GEGGI8GHGCG=D4GE@GHGG
chr9	106056048	N	32	<<<<<<<<<GGGGGGGGGGGGGGGGGGGGGGG	BHEIG?DIIG9GDGGIGGHEFGBG<GDGBHGG


이제 pandas에 불러와서 분석해 봅니다. R을 쓰시는 분들은 R로 보내서 작업하셔도 됩니다.

In [112]:
import pandas as pd

pileup = pd.read_csv('CLIP-let7g-gene.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])
pileup.head()

,chrom,pos,_ref,count,basereads,quals
0,chr9,106056039,N,9,<<<<<<<<<,BHEIG?DII
1,chr9,106056040,N,9,<<<<<<<<<,BHEIG?DII
2,chr9,106056041,N,9,<<<<<<<<<,BHEIG?DII
3,chr9,106056042,N,9,<<<<<<<<<,BHEIG?DII
4,chr9,106056043,N,9,<<<<<<<<<,BHEIG?DII


여기서 pileup에서 계산에 사용할 것은 match와 substitution만입니다. 나머지에 해당하는 tag들은 제거합니다.

R을 쓰시는 분들은 R로 보내서 작업하셔도 됩니다.

In [113]:
import re
toremove = re.compile('[<>$*#^]')
pileup['matches'] = pileup['basereads'].apply(lambda x: toremove.sub('', x))

In [114]:
pileup[['chrom', 'pos', 'matches']]

,chrom,pos,matches
0,chr9,106056039,
1,chr9,106056040,
2,chr9,106056041,
3,chr9,106056042,
4,chr9,106056043,
...,...,...,...
83,chr9,106056122,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
84,chr9,106056123,CCCCCCCCCCCCCCCCCCCCCC
85,chr9,106056124,AAAAAAAAAAAAAAAAAAAAAA
86,chr9,106056125,GGGGGGGGGGGGGGGGGGGGGG


In [115]:
pileup[pileup['pos'] == 106056094].iloc[0]['matches']

'GGGGGGAAAAAAAAGGGGGAAAAAAGCCGCAGGATGAGGTGATAAGGGAGGGGTGAAGGGCGGTGAAGGGGAAAAGAGAAAGAAAAATAAAGGGGGAGTGGGAGGAAGAAGAGAATA'

In [116]:
pileup

,chrom,pos,_ref,count,basereads,quals,matches
0,chr9,106056039,N,9,<<<<<<<<<,BHEIG?DII,
1,chr9,106056040,N,9,<<<<<<<<<,BHEIG?DII,
2,chr9,106056041,N,9,<<<<<<<<<,BHEIG?DII,
3,chr9,106056042,N,9,<<<<<<<<<,BHEIG?DII,
4,chr9,106056043,N,9,<<<<<<<<<,BHEIG?DII,
...,...,...,...,...,...,...,...
83,chr9,106056122,N,88,<<<<<<<<<C$C$C$C$C$C$C$C$C$C$C$C$C$CCCCCC$C$C$...,BHEIG?DIIEEGIIC;GIHEGBIIIIB1=FII?FEIGGGHDBIG=H...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
84,chr9,106056123,N,31,<<<<<<<<<CCCCCCCCCCCCCCCCCCCCCC,BHEIG?DIIIIII>GIGGIGGD>BIHHHIEH,CCCCCCCCCCCCCCCCCCCCCC
85,chr9,106056124,N,31,<<<<<<<<<AAAAAAAAAAAAAAAAAAAAAA,BHEIG?DIIIIHIGGIGGGIG:9DDBIEGFH,AAAAAAAAAAAAAAAAAAAAAA
86,chr9,106056125,N,31,<<<<<<<<<GGGGGGGGGGGGGGGGGGGGGG,BHEIG?DIIIIIIGGE@GFIGD;GIGIIFHD,GGGGGGGGGGGGGGGGGGGGGG


자 이제 데이터 준비가 대충 끝났습니다. 다음 순서로 진행해 보세요. (1-4번 단계는 R을 쓰시는 분들은 R로 보내서 작업하셔도 됩니다.)

1. 각 position별로 base수를 셉니다.
1. 각 position별로 Shannon entropy를 계산합니다.
1. 계산한 Shannon entropy를 [bedgraph format](https://genome.ucsc.edu/goldenPath/help/bedgraph.html)으로 출력합니다. 말은 복잡해도 실제로는 간단합니다. 4칸짜리를 만드시면 됩니다.
1. 결과 파일을 본인의 Google Drive에서 가져옵니다.
1. [UCSC Genome Browser](http://genome.ucsc.edu/cgi-bin/hgTracks?db=mm39&lastVirtModeType=default&lastVirtModeExtraState=&virtModeType=default&virtMode=0&nonVirtPosition=&position=chr9%3A106056039-106056126)에 접속해서 Genome은 mm39로 선택합니다.
1. 그래프 아랫쪽의 add custom tracks 버튼을 누릅니다.
1. Paste URLs or data 옆의 Choose File 버튼을 누르고 우리가 만든 bedgraph 파일을 업로드 합니다.
1. 그리고 이리 저리 감상해 보다가 View 메뉴의 PDF/PS 를 눌러서 인증샷을 한 번 찍습니다.
1. Mirlet7d와 Mirlet7f-1도 한 번 해 봅니다.

In [117]:
pileup[pileup.pos == 106056039].matches

0    
Name: matches, dtype: object

In [118]:
for base in ["A", "T", "C", "G"]:
    pileup[base] = pileup.matches.str.count(base)

pileup

,chrom,pos,_ref,count,basereads,quals,matches,A,T,C,G
0,chr9,106056039,N,9,<<<<<<<<<,BHEIG?DII,,0,0,0,0
1,chr9,106056040,N,9,<<<<<<<<<,BHEIG?DII,,0,0,0,0
2,chr9,106056041,N,9,<<<<<<<<<,BHEIG?DII,,0,0,0,0
3,chr9,106056042,N,9,<<<<<<<<<,BHEIG?DII,,0,0,0,0
4,chr9,106056043,N,9,<<<<<<<<<,BHEIG?DII,,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
83,chr9,106056122,N,88,<<<<<<<<<C$C$C$C$C$C$C$C$C$C$C$C$C$CCCCCC$C$C$...,BHEIG?DIIEEGIIC;GIHEGBIIIIB1=FII?FEIGGGHDBIG=H...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,0,0,79,0
84,chr9,106056123,N,31,<<<<<<<<<CCCCCCCCCCCCCCCCCCCCCC,BHEIG?DIIIIII>GIGGIGGD>BIHHHIEH,CCCCCCCCCCCCCCCCCCCCCC,0,0,22,0
85,chr9,106056124,N,31,<<<<<<<<<AAAAAAAAAAAAAAAAAAAAAA,BHEIG?DIIIIHIGGIGGGIG:9DDBIEGFH,AAAAAAAAAAAAAAAAAAAAAA,22,0,0,0
86,chr9,106056125,N,31,<<<<<<<<<GGGGGGGGGGGGGGGGGGGGGG,BHEIG?DIIIIIIGGE@GFIGD;GIGIIFHD,GGGGGGGGGGGGGGGGGGGGGG,0,0,0,22


In [119]:
pileup[["A", "T", "C", "G"]] = pileup[["A", "T", "C", "G"]].div(pileup[["A", "T", "C", "G"]].sum(axis=1), axis=0)
pileup = pileup.dropna()
pileup

,chrom,pos,_ref,count,basereads,quals,matches,A,T,C,G
6,chr9,106056045,N,32,<<<<<<<<<^HT^DT^IT^GT^HT^HT^IT^GT^IT^HT^HT^IT^...,BHEIG?DIIG9GEGGIDGEGFGBG;DDGHDGG,HTDTITGTHTHTITGTITHTHTITITHTHTGTHTGTHTHTHTITGT,0.0,0.821429,0.0,0.178571
7,chr9,106056046,N,32,<<<<<<<<<GGGGGGGGGGGGGGGGGGGGGGG,BHEIG?DIIG<GGGGIGGBGFG?G?GEGHHGG,GGGGGGGGGGGGGGGGGGGGGGG,0.0,0.000000,0.0,1.000000
8,chr9,106056047,N,32,<<<<<<<<<AAAAAAAAAAAAAAAAAAAAAAA,BHEIG?DIIG7GEGGI8GHGCG=D4GE@GHGG,AAAAAAAAAAAAAAAAAAAAAAA,1.0,0.000000,0.0,0.000000
9,chr9,106056048,N,32,<<<<<<<<<GGGGGGGGGGGGGGGGGGGGGGG,BHEIG?DIIG9GDGGIGGHEFGBG<GDGBHGG,GGGGGGGGGGGGGGGGGGGGGGG,0.0,0.000000,0.0,1.000000
10,chr9,106056049,N,32,<<<<<<<<<GGGGGGGGGGGGGGGGGGGGGGG,BHEIG?DIIG:GGGGIDGHGFG?G:GDEEHGB,GGGGGGGGGGGGGGGGGGGGGGG,0.0,0.000000,0.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
83,chr9,106056122,N,88,<<<<<<<<<C$C$C$C$C$C$C$C$C$C$C$C$C$CCCCCC$C$C$...,BHEIG?DIIEEGIIC;GIHEGBIIIIB1=FII?FEIGGGHDBIG=H...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,0.0,0.000000,1.0,0.000000
84,chr9,106056123,N,31,<<<<<<<<<CCCCCCCCCCCCCCCCCCCCCC,BHEIG?DIIIIII>GIGGIGGD>BIHHHIEH,CCCCCCCCCCCCCCCCCCCCCC,0.0,0.000000,1.0,0.000000
85,chr9,106056124,N,31,<<<<<<<<<AAAAAAAAAAAAAAAAAAAAAA,BHEIG?DIIIIHIGGIGGGIG:9DDBIEGFH,AAAAAAAAAAAAAAAAAAAAAA,1.0,0.000000,0.0,0.000000
86,chr9,106056125,N,31,<<<<<<<<<GGGGGGGGGGGGGGGGGGGGGG,BHEIG?DIIIIIIGGE@GFIGD;GIGIIFHD,GGGGGGGGGGGGGGGGGGGGGG,0.0,0.000000,0.0,1.000000


In [120]:
import numpy as np
from scipy.stats import entropy

prop = pileup[["A", "T", "C", "G"]].to_numpy()
pileup["entropy"] = -np.sum(prop * np.log(prop + 1e-9), axis=1)
pileup

/data/packages/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,chrom,pos,_ref,count,basereads,quals,matches,A,T,C,G,entropy
6,chr9,106056045,N,32,<<<<<<<<<^HT^DT^IT^GT^HT^HT^IT^GT^IT^HT^HT^IT^...,BHEIG?DIIG9GEGGIDGEGFGBG;DDGHDGG,HTDTITGTHTHTITGTITHTHTITITHTHTGTHTGTHTHTHTITGT,0.0,0.821429,0.0,0.178571,4.692203e-01
7,chr9,106056046,N,32,<<<<<<<<<GGGGGGGGGGGGGGGGGGGGGGG,BHEIG?DIIG<GGGGIGGBGFG?G?GEGHHGG,GGGGGGGGGGGGGGGGGGGGGGG,0.0,0.000000,0.0,1.000000,-1.000000e-09
8,chr9,106056047,N,32,<<<<<<<<<AAAAAAAAAAAAAAAAAAAAAAA,BHEIG?DIIG7GEGGI8GHGCG=D4GE@GHGG,AAAAAAAAAAAAAAAAAAAAAAA,1.0,0.000000,0.0,0.000000,-1.000000e-09
9,chr9,106056048,N,32,<<<<<<<<<GGGGGGGGGGGGGGGGGGGGGGG,BHEIG?DIIG9GDGGIGGHEFGBG<GDGBHGG,GGGGGGGGGGGGGGGGGGGGGGG,0.0,0.000000,0.0,1.000000,-1.000000e-09
10,chr9,106056049,N,32,<<<<<<<<<GGGGGGGGGGGGGGGGGGGGGGG,BHEIG?DIIG:GGGGIDGHGFG?G:GDEEHGB,GGGGGGGGGGGGGGGGGGGGGGG,0.0,0.000000,0.0,1.000000,-1.000000e-09
...,...,...,...,...,...,...,...,...,...,...,...,...
83,chr9,106056122,N,88,<<<<<<<<<C$C$C$C$C$C$C$C$C$C$C$C$C$CCCCCC$C$C$...,BHEIG?DIIEEGIIC;GIHEGBIIIIB1=FII?FEIGGGHDBIG=H...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,0.0,0.000000,1.0,0.000000,-1.000000e-09
84,chr9,106056123,N,31,<<<<<<<<<CCCCCCCCCCCCCCCCCCCCCC,BHEIG?DIIIIII>GIGGIGGD>BIHHHIEH,CCCCCCCCCCCCCCCCCCCCCC,0.0,0.000000,1.0,0.000000,-1.000000e-09
85,chr9,106056124,N,31,<<<<<<<<<AAAAAAAAAAAAAAAAAAAAAA,BHEIG?DIIIIHIGGIGGGIG:9DDBIEGFH,AAAAAAAAAAAAAAAAAAAAAA,1.0,0.000000,0.0,0.000000,-1.000000e-09
86,chr9,106056125,N,31,<<<<<<<<<GGGGGGGGGGGGGGGGGGGGGG,BHEIG?DIIIIIIGGE@GFIGD;GIGIIFHD,GGGGGGGGGGGGGGGGGGGGGG,0.0,0.000000,0.0,1.000000,-1.000000e-09


In [121]:
pileup = pileup[["chrom", "pos", "pos", "entropy"]]
pileup.columns = ["chrom", "start", "end", "entropy"]
pileup["start"] = pileup.start - 1
pileup

/data/packages/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,chrom,start,end,entropy
6,chr9,106056044,106056045,4.692203e-01
7,chr9,106056045,106056046,-1.000000e-09
8,chr9,106056046,106056047,-1.000000e-09
9,chr9,106056047,106056048,-1.000000e-09
10,chr9,106056048,106056049,-1.000000e-09
...,...,...,...,...
83,chr9,106056121,106056122,-1.000000e-09
84,chr9,106056122,106056123,-1.000000e-09
85,chr9,106056123,106056124,-1.000000e-09
86,chr9,106056124,106056125,-1.000000e-09


In [122]:
pileup.to_csv("let7g.bed", sep="\t", header=None, index=False)